# 1. Import the libraries 

In [2]:
#import the reqired libraries
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
# from sklearn.externals import joblib
import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

# 2. Load the dataset and add headers

In [40]:
# Import the dataset and give the column names
columns=["disability_type","skills_one", "skills_two", "position", "is_hearing_impairment", "is_visual_impairment", "is_physical_disability", "is_all_types_of_disabilities", 
              "is_neurodiversity", "is_mobility_impairment", "is_speech_impairment", "is_cognitive_disability", "is_learning_disability"]
df=pd.read_csv('C:/Users/LENOVO/Downloads/DatasetClearRichard.csv',names=None)

In [41]:
df.head()

,disability_type,skills_one,skills_two,company,city,province,job_type,position,is_hearing_impairment,is_visual_impairment,is_physical_disability,is_all_types_of_disabilities,is_neurodiversity,is_mobility_impairment,is_speech_impairment,is_cognitive_disability,is_learning_disability
0,Visual Impairment,Communication skills,Microsoft Excel,ABC Corporation,Jakarta,Jakarta Raya,Full-time,Data Analyst,0,1,0,0,0,0,0,0,0
1,Hearing Impairment,Teamwork,Python,XYZ Solutions,Bandung,Jawa Barat,Contract,Software Developer,1,0,0,0,0,0,0,0,0
2,Physical Disability,Leadership,Java,Acme Technologies,Surabaya,Jawa Timur,Part-time,IT Support Specialist,0,0,1,0,0,0,0,0,0
3,Learning Disability,Time management,HTML,Global Ventures,Medan,Sumatera Utara,Full-time,Web Designer,0,0,0,0,0,0,0,0,1
4,Cognitive Disability,Analytical skills,C++,Innovate Systems,Semarang,Jawa Tengah,Contract,Software Tester,0,0,0,0,0,0,0,1,0


#### Dropping the timestamp column

In [42]:
df.drop('disability_type',axis=1,inplace=True)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006 entries, 0 to 1005
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   skills_one                    1006 non-null   object
 1   skills_two                    1006 non-null   object
 2   company                       1006 non-null   object
 3   city                          1006 non-null   object
 4   province                      1005 non-null   object
 5   job_type                      1006 non-null   object
 6   position                      1006 non-null   object
 7   is_hearing_impairment         1006 non-null   int64 
 8   is_visual_impairment          1006 non-null   int64 
 9   is_physical_disability        1006 non-null   int64 
 10  is_all_types_of_disabilities  1006 non-null   int64 
 11  is_neurodiversity             1006 non-null   int64 
 12  is_mobility_impairment        1006 non-null   int64 
 13  is_speech_impairme

In [44]:
#Check the number of rows and columns
rows,columns=df.shape
print('Number of rows: ',rows)
print('Number of columns: ',columns)

Number of rows:  1006
Number of columns:  16


In [45]:
#Check the datatypes
df.dtypes

skills_one                      object
skills_two                      object
company                         object
city                            object
province                        object
job_type                        object
position                        object
is_hearing_impairment            int64
is_visual_impairment             int64
is_physical_disability           int64
is_all_types_of_disabilities     int64
is_neurodiversity                int64
is_mobility_impairment           int64
is_speech_impairment             int64
is_cognitive_disability          int64
is_learning_disability           int64
dtype: object

In [46]:
#Taking subset of the dataset
df1=df.iloc[:50000,0:]

* Since the data is very big. Consider electronics_df1 named dataframe with  first 50000 rows and all columns from 0 of dataset.

In [47]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006 entries, 0 to 1005
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   skills_one                    1006 non-null   object
 1   skills_two                    1006 non-null   object
 2   company                       1006 non-null   object
 3   city                          1006 non-null   object
 4   province                      1005 non-null   object
 5   job_type                      1006 non-null   object
 6   position                      1006 non-null   object
 7   is_hearing_impairment         1006 non-null   int64 
 8   is_visual_impairment          1006 non-null   int64 
 9   is_physical_disability        1006 non-null   int64 
 10  is_all_types_of_disabilities  1006 non-null   int64 
 11  is_neurodiversity             1006 non-null   int64 
 12  is_mobility_impairment        1006 non-null   int64 
 13  is_speech_impairme

In [48]:
#Summary statistics of rating variable
df1['skills'] = df['skills_one'] + ' ' + df['skills_two']
df1['skills'].describe().transpose()

count                     1006
unique                      34
top       Time management HTML
freq                       116
Name: skills, dtype: object

In [49]:
#Find the minimum and maximum ratings
# print('Minimum rating is: %d' %(electronics_df1.ratings.min()))
print('Recommended skill is: %s' %(df1.skills))

Recommended skill is: 0       Communication skills Microsoft Excel
1                            Teamwork Python
2                            Leadership Java
3                       Time management HTML
4                      Analytical skills C++
                        ...                 
1001              Attention to detail Python
1002    Communication skills Microsoft Excel
1003                    Time management HTML
1004         Critical thinking Ruby on Rails
1005                   Analytical skills C++
Name: skills, Length: 1006, dtype: object


* Rating are on the scale 1 to 5.

# Handling Missing values

In [50]:
#Check for missing values
print('Number of missing values across columns: \n',df.isnull().sum())

Number of missing values across columns: 
 skills_one                      0
skills_two                      0
company                         0
city                            0
province                        1
job_type                        0
position                        0
is_hearing_impairment           0
is_visual_impairment            0
is_physical_disability          0
is_all_types_of_disabilities    0
is_neurodiversity               0
is_mobility_impairment          0
is_speech_impairment            0
is_cognitive_disability         0
is_learning_disability          0
dtype: int64


* There are no missing records in the dataset.

## Ratings

* We can see that more number of users have given the rating of 5.

## Users and products

In [51]:
# Number of unique user id  in the data
print('Number of unique skill first users in Raw data = ', df1['skills_one'].nunique())
# Number of unique product id  in the data
print('Number of unique skill second users in Raw data = ', df1['skills_two'].nunique())

print('Number of unique skill overall users in Raw data = ', df1['skills'].nunique())

Number of unique skill first users in Raw data =  20
Number of unique skill second users in Raw data =  22
Number of unique skill overall users in Raw data =  34


# 3. Taking the subset of dataset to make it less sparse/ denser.

In [52]:
#Check the top 10 users based on ratings
most_rated=df1.groupby('skills').size().sort_values(ascending=False)[:10]
print('Top 10 skills based on ratings: \n',most_rated)

Top 10 skills based on ratings: 
 skills
Time management HTML                    116
Attention to detail Python              110
Collaboration SQL                        88
Critical thinking Ruby on Rails          88
Analytical skills C++                    80
Communication skills Microsoft Excel     70
Leadership Java                          61
Adaptability PHP                         48
Critical thinking PHP                    44
Problem-solving JavaScript               42
dtype: int64


In [53]:
counts=df1.skills.value_counts()
df1_final=df1[df1.skills.isin(counts[counts>=15].index)]
# print('Number of users who have rated 25 or more items =', len(df1_final))
print('Number of unique skills in the final data = ', df1_final['skills'].nunique())
# print('Number of unique products in the final data = ', df1_final['userId'].nunique())

Number of unique skills in the final data =  16


# 4. Splitting the data

In [54]:
#Split the data randomnly into train and test datasets into 70:30 ratio
train_data, test_data = train_test_split(df1_final, test_size = 0.3, random_state=0)
train_data.head()

,skills_one,skills_two,company,city,province,job_type,position,is_hearing_impairment,is_visual_impairment,is_physical_disability,is_all_types_of_disabilities,is_neurodiversity,is_mobility_impairment,is_speech_impairment,is_cognitive_disability,is_learning_disability,skills
118,Leadership,JavaScript,ByteTech,Surabaya,Jawa Timur,Part-time,Web Developer,0,1,0,0,0,0,0,0,0,Leadership JavaScript
78,Attention to detail,Python,TechConnect,Jakarta,Jakarta Raya,Part-time,Data Engineer,1,0,0,0,0,0,0,0,0,Attention to detail Python
50,Communication skills,Microsoft Excel,Innovation Hub,Jakarta,Jakarta Raya,Full-time,Administrative Assistant,0,1,0,0,0,0,0,0,0,Communication skills Microsoft Excel
609,Communication skills,Microsoft Office,Innovation Hub,Jakarta,Jakarta Raya,Full-time,Web Designer,0,1,0,0,0,0,0,0,0,Communication skills Microsoft Office
320,Collaboration,SQL,CyberTech,Surabaya,Jawa Timur,Full-time,Software Engineer,0,0,1,0,0,0,0,0,0,Collaboration SQL


In [55]:
print('Shape of training data: ',train_data.shape)
print('Shape of testing data: ',test_data.shape)

Shape of training data:  (658, 17)
Shape of testing data:  (283, 17)


# 5. Building Popularity Recommder model

In [57]:
#Count of user_id for each unique product as recommendation score 
# train_data_grouped = train_data.groupby('skills_one').agg({'userId': 'count'}).reset_index()
train_data_grouped = train_data.groupby('skills_one')
# train_data_grouped.rename(columns = {'skills_one': 'score'},inplace=True)
train_data_grouped.head(40)

,skills_one,skills_two,company,city,province,job_type,position,is_hearing_impairment,is_visual_impairment,is_physical_disability,is_all_types_of_disabilities,is_neurodiversity,is_mobility_impairment,is_speech_impairment,is_cognitive_disability,is_learning_disability,skills
118,Leadership,JavaScript,ByteTech,Surabaya,Jawa Timur,Part-time,Web Developer,0,1,0,0,0,0,0,0,0,Leadership JavaScript
78,Attention to detail,Python,TechConnect,Jakarta,Jakarta Raya,Part-time,Data Engineer,1,0,0,0,0,0,0,0,0,Attention to detail Python
50,Communication skills,Microsoft Excel,Innovation Hub,Jakarta,Jakarta Raya,Full-time,Administrative Assistant,0,1,0,0,0,0,0,0,0,Communication skills Microsoft Excel
609,Communication skills,Microsoft Office,Innovation Hub,Jakarta,Jakarta Raya,Full-time,Web Designer,0,1,0,0,0,0,0,0,0,Communication skills Microsoft Office
320,Collaboration,SQL,CyberTech,Surabaya,Jawa Timur,Full-time,Software Engineer,0,0,1,0,0,0,0,0,0,Collaboration SQL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Problem-solving,JavaScript,ByteBytes,Yogyakarta,Yogyakarta,Full-time,UX Designer,1,0,0,0,0,0,0,0,0,Problem-solving JavaScript
613,Problem-solving,UI/UX design,WebCraft,Yogyakarta,Yogyakarta,Full-time,Web Designer,0,1,0,0,0,0,0,0,0,Problem-solving UI/UX design
702,Problem-solving,UI/UX design,WebCraft,Yogyakarta,Yogyakarta,Full-time,Administrative Assistant,0,1,0,0,0,0,0,0,0,Problem-solving UI/UX design
768,Teamwork,Python,XYZ Solutions,Bandung,Jawa Barat,Contract,Graphic Designer,1,0,0,0,0,0,0,0,0,Teamwork Python


In [58]:
#Sort the products on recommendation score 
train_data_sort = train_data_grouped.sort_values(['skills_one', 'skills_two'], ascending = [0,1]) 
      
#Generate a recommendation rank based upon score 
train_data_sort['skills_one'] = train_data_sort['skills_two'].rank(ascending=0, method='first') 
          
#Get the top 5 recommendations 
popularity_recommendations = train_data_sort.head(5) 
popularity_recommendations 

AttributeError: 'DataFrameGroupBy' object has no attribute 'sort_values'

In [24]:
# Use popularity based recommender model to make predictions
def recommend(user_id):     
    user_recommendations = popularity_recommendations 
          
    #Add user_id column for which the recommendations are being generated 
    user_recommendations['userId'] = user_id 
      
    #Bring user_id column to the front 
    cols = user_recommendations.columns.tolist() 
    cols = cols[-1:] + cols[:-1] 
    user_recommendations = user_recommendations[cols] 
          
    return user_recommendations 

In [25]:
find_recom = [10,100,150]   # This list is user choice.
for i in find_recom:
    print("The list of recommendations for the userId: %d\n" %(i))
    print(recommend(i))    
    print("\n") 

The list of recommendations for the userId: 10

     userId   productId  score  rank
87       10  B00004RC2D      4   1.0
57       10  B00002SWHH      3   2.0
99       10  B00004SC3Y      3   3.0
100      10  B00004SCKA      3   4.0
6        10  B00000J3II      2   5.0


The list of recommendations for the userId: 100

     userId   productId  score  rank
87      100  B00004RC2D      4   1.0
57      100  B00002SWHH      3   2.0
99      100  B00004SC3Y      3   3.0
100     100  B00004SCKA      3   4.0
6       100  B00000J3II      2   5.0


The list of recommendations for the userId: 150

     userId   productId  score  rank
87      150  B00004RC2D      4   1.0
57      150  B00002SWHH      3   2.0
99      150  B00004SC3Y      3   3.0
100     150  B00004SCKA      3   4.0
6       150  B00000J3II      2   5.0




* Since, it is a Popularity recommender model, so, all the three users are given the same recommendations. Here, we predict the products based on the popularity. It is not personalized to particular user. It is a non-personalized recommender system.

# 6. Building Collaborative Filtering recommender model.

In [26]:
electronics_df_CF = pd.concat([train_data, test_data]).reset_index()
electronics_df_CF.head()

,index,userId,productId,ratings
0,17509,AY8Q1X7G96HV5,B00000JSES,4.0
1,11968,A243HY69GIAHFI,B00000J3Q7,3.0
2,35533,A1RPTVW5VEOSI,B00003WGP5,5.0
3,31480,A1NVD0TKNS1GT5,B00002JXFH,4.0
4,13526,A23ZO1BVFFLGHO,B00000J570,5.0


## User Based Collaborative Filtering model

In [27]:
# Matrix with row per 'user' and column per 'item' 
pivot_df = electronics_df_CF.pivot(index = 'userId', columns ='productId', values = 'ratings').fillna(0)
pivot_df.head()

productId,1400599997,B00000DM9M,B00000J061,B00000J08C,B00000J0A2,B00000J0E8,B00000J1QZ,B00000J1US,B00000J3H5,B00000J3HB,...,B00004TDWY,B00004TE75,B00004TEN2,B00004TH2W,B00004TH2Y,B00004THCX,B00004THCY,B00004THCZ,B00004THDE,B00004THM6
userId,,,,,,,,,,,,,,,,,,,,,
A1ISUNUWG0K02V,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1MJMYLRTZ76ZX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1NVD0TKNS1GT5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1RPTVW5VEOSI,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
A231WM2Z2JL0U3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,5.0,5.0,0.0,4.0,0.0,0.0,0.0,5.0,0.0


In [28]:
print('Shape of the pivot table: ', pivot_df.shape)

Shape of the pivot table:  (11, 186)


In [29]:
#define user index from 0 to 10
pivot_df['user_index'] = np.arange(0, pivot_df.shape[0], 1)
pivot_df.head()

productId,1400599997,B00000DM9M,B00000J061,B00000J08C,B00000J0A2,B00000J0E8,B00000J1QZ,B00000J1US,B00000J3H5,B00000J3HB,...,B00004TE75,B00004TEN2,B00004TH2W,B00004TH2Y,B00004THCX,B00004THCY,B00004THCZ,B00004THDE,B00004THM6,user_index
userId,,,,,,,,,,,,,,,,,,,,,
A1ISUNUWG0K02V,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
A1MJMYLRTZ76ZX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
A1NVD0TKNS1GT5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
A1RPTVW5VEOSI,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,3
A231WM2Z2JL0U3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,5.0,5.0,0.0,4.0,0.0,0.0,0.0,5.0,0.0,4


In [30]:
pivot_df.set_index(['user_index'], inplace=True)
# Actual ratings given by users
pivot_df.head()

productId,1400599997,B00000DM9M,B00000J061,B00000J08C,B00000J0A2,B00000J0E8,B00000J1QZ,B00000J1US,B00000J3H5,B00000J3HB,...,B00004TDWY,B00004TE75,B00004TEN2,B00004TH2W,B00004TH2Y,B00004THCX,B00004THCY,B00004THCZ,B00004THDE,B00004THM6
user_index,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,5.0,5.0,0.0,4.0,0.0,0.0,0.0,5.0,0.0


* As this is a sparse matrix we will use SVD.

### Singular Value Decomposition

In [31]:
# Singular Value Decomposition
U, sigma, Vt = svds(pivot_df, k = 10)

In [32]:
print('Left singular matrix: \n',U)

Left singular matrix: 
 [[ 2.65871898e-01  8.28788249e-01  1.14160900e-01 -2.15685919e-01
  -2.65928896e-04 -4.93636653e-02  1.66297884e-01 -3.12732051e-01
  -1.53122256e-01  1.38465143e-01]
 [ 5.53711304e-02  1.21204397e-01 -8.17836609e-02  3.29682359e-01
  -1.47955175e-01 -8.18019500e-01 -2.13209931e-01  2.76257471e-01
  -2.09061119e-01  1.01498819e-01]
 [ 5.46679471e-01 -1.37512688e-01  2.79596983e-02  1.45212122e-01
  -3.57798248e-02  1.89642626e-01 -6.60636300e-01 -2.07100247e-01
   3.59790632e-02  1.23044532e-01]
 [ 4.63168744e-02  5.39880164e-02 -8.32935752e-01  1.36292972e-01
   1.12974956e-02  9.74799278e-02 -5.43326589e-02 -2.99913518e-01
  -1.00398248e-01  1.16155900e-01]
 [-4.82857184e-02 -5.06969281e-03  6.10478781e-02  7.63553438e-02
   8.79528911e-02  1.33412578e-01  9.24837886e-02  2.48586367e-01
   1.79064339e-01  9.26201600e-01]
 [-1.33072419e-01 -3.30520700e-01 -8.58433108e-03 -6.82184355e-01
  -3.10981905e-02 -4.06414738e-01 -1.22417282e-01 -4.14339191e-01
   4.0445

In [33]:
print('Sigma: \n',sigma)

Sigma: 
 [15.65710264 16.05091576 17.43121071 17.74252629 18.18622003 18.83061496
 19.14527912 19.94315139 22.99328895 28.83750492]


* As sigma is not a diagonal matrix we have to convert it into diagonal matrix.

In [34]:
# Construct diagonal array in SVD
sigma = np.diag(sigma)
print('Diagonal matrix: \n',sigma)

Diagonal matrix: 
 [[15.65710264  0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.         16.05091576  0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.         17.43121071  0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         17.74252629  0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.         18.18622003  0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.         18.83061496
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
  19.14527912  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.         19.94315139  0.          0.        ]
 [ 0.          0.          0.        

In [35]:
print('Right singular matrix: \n',Vt)

Right singular matrix: 
 [[-0.00496595 -0.01376713  0.01479101 ... -0.0110137  -0.01541975
  -0.0110137 ]
 [-0.01476518 -0.04204683  0.01681774 ... -0.03363746 -0.00157925
  -0.03363746]
 [ 0.01587168  0.01649817 -0.2389208  ...  0.01319853  0.01751108
   0.01319853]
 ...
 [-0.0315406   0.04129911 -0.07519211 ...  0.03303929  0.06232374
   0.03303929]
 [-0.0067028  -0.20224783 -0.02183208 ... -0.16179826  0.03893839
  -0.16179826]
 [ 0.00421311  0.01858071  0.02013973 ...  0.01486457  0.16058976
   0.01486457]]


In [36]:
#Predicted ratings
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
# Convert predicted ratings to dataframe
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = pivot_df.columns)
preds_df.head()

productId,1400599997,B00000DM9M,B00000J061,B00000J08C,B00000J0A2,B00000J0E8,B00000J1QZ,B00000J1US,B00000J3H5,B00000J3HB,...,B00004TDWY,B00004TE75,B00004TEN2,B00004TH2W,B00004TH2Y,B00004THCX,B00004THCY,B00004THCZ,B00004THDE,B00004THM6
0,-0.005077,-0.010023,0.219864,-0.010023,-0.020310,-0.006562,2.962929,0.026594,4.938216,4.938216,...,-0.025387,0.026594,0.026594,0.219864,0.021275,-0.008018,-0.006014,-0.008018,0.026594,-0.008018
1,-0.000261,-0.000515,0.011300,-0.000515,-0.001044,-0.000337,-0.001905,0.001367,-0.003176,-0.003176,...,-0.001305,0.001367,0.001367,0.011300,0.001093,-0.000412,-0.000309,-0.000412,0.001367,-0.000412
2,-0.016300,-0.032177,0.705839,-0.032177,-0.065201,-0.021066,-0.119009,0.085374,-0.198349,-0.198349,...,-0.081502,0.085374,0.085374,0.705839,0.068299,-0.025742,-0.019306,-0.025742,0.085374,-0.025742
3,0.018068,0.035667,4.217599,0.035667,0.072274,0.023351,0.131918,-0.094635,0.219864,0.219864,...,0.090342,-0.094635,-0.094635,4.217599,-0.075708,0.028534,0.021400,0.028534,-0.094635,0.028534
4,0.002185,0.004314,-0.094635,0.004314,0.008742,0.002824,0.015956,4.988553,0.026594,0.026594,...,0.010927,4.988553,4.988553,-0.094635,3.990843,0.003451,0.002588,0.003451,4.988553,0.003451


In [37]:
# Recommend the items with the highest predicted ratings

def recommend_items(userID, pivot_df, preds_df, num_recommendations):
    # index starts at 0  
    user_idx = userID-1 
    # Get and sort the user's ratings
    sorted_user_ratings = pivot_df.iloc[user_idx].sort_values(ascending=False)
    #sorted_user_ratings
    sorted_user_predictions = preds_df.iloc[user_idx].sort_values(ascending=False)
    #sorted_user_predictions
    temp = pd.concat([sorted_user_ratings, sorted_user_predictions], axis=1)
    temp.index.name = 'Recommended Items'
    temp.columns = ['user_ratings', 'user_predictions']
    temp = temp.loc[temp.user_ratings == 0]   
    temp = temp.sort_values('user_predictions', ascending=False)
    print('\nBelow are the recommended items for user(user_id = {}):\n'.format(userID))
    print(temp.head(num_recommendations))

In [38]:
userID = 4
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)    


Below are the recommended items for user(user_id = 4):

                   user_ratings  user_predictions
Recommended Items                                
B00000JYLO                  0.0          1.526746
B00001P4XA                  0.0          1.526746
B00004T1WZ                  0.0          1.526746
B00004T1WX                  0.0          1.526746
B00000J4ER                  0.0          1.526746


In [39]:
userID = 6
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)


Below are the recommended items for user(user_id = 6):

                   user_ratings  user_predictions
Recommended Items                                
B00004S9WQ                  0.0          0.038989
B00000J061                  0.0          0.038989
B00004RIPE                  0.0          0.038989
B00004RERZ                  0.0          0.038989
B00001QHP5                  0.0          0.038989


In [40]:
userID = 8
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)


Below are the recommended items for user(user_id = 8):

                   user_ratings  user_predictions
Recommended Items                                
B00004TH2W                  0.0          1.526746
B00004RERZ                  0.0          1.526746
B00003WGP5                  0.0          1.526746
B00000J061                  0.0          1.526746
B00004RIPE                  0.0          1.526746


* Since, it is a Collaborative recommender model, so, all the three users are given different recommendations based on users past behaviour.

# 7. Evaluation of Collabrative recommendation model 

In [41]:
# Actual ratings given by the users
final_ratings_matrix.head()

productId,1400599997,B00000DM9M,B00000J061,B00000J08C,B00000J0A2,B00000J0E8,B00000J1QZ,B00000J1US,B00000J3H5,B00000J3HB,...,B00004TDWY,B00004TE75,B00004TEN2,B00004TH2W,B00004TH2Y,B00004THCX,B00004THCY,B00004THCZ,B00004THDE,B00004THM6
userId,,,,,,,,,,,,,,,,,,,,,
A1ISUNUWG0K02V,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1MJMYLRTZ76ZX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1NVD0TKNS1GT5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1RPTVW5VEOSI,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
A231WM2Z2JL0U3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,5.0,5.0,0.0,4.0,0.0,0.0,0.0,5.0,0.0


In [42]:
# Average ACTUAL rating for each item
final_ratings_matrix.mean().head()

productId
1400599997    0.090909
B00000DM9M    0.454545
B00000J061    0.454545
B00000J08C    0.454545
B00000J0A2    0.363636
dtype: float64

In [43]:
# Predicted ratings 
preds_df.head()

productId,1400599997,B00000DM9M,B00000J061,B00000J08C,B00000J0A2,B00000J0E8,B00000J1QZ,B00000J1US,B00000J3H5,B00000J3HB,...,B00004TDWY,B00004TE75,B00004TEN2,B00004TH2W,B00004TH2Y,B00004THCX,B00004THCY,B00004THCZ,B00004THDE,B00004THM6
0,-0.005077,-0.010023,0.219864,-0.010023,-0.020310,-0.006562,2.962929,0.026594,4.938216,4.938216,...,-0.025387,0.026594,0.026594,0.219864,0.021275,-0.008018,-0.006014,-0.008018,0.026594,-0.008018
1,-0.000261,-0.000515,0.011300,-0.000515,-0.001044,-0.000337,-0.001905,0.001367,-0.003176,-0.003176,...,-0.001305,0.001367,0.001367,0.011300,0.001093,-0.000412,-0.000309,-0.000412,0.001367,-0.000412
2,-0.016300,-0.032177,0.705839,-0.032177,-0.065201,-0.021066,-0.119009,0.085374,-0.198349,-0.198349,...,-0.081502,0.085374,0.085374,0.705839,0.068299,-0.025742,-0.019306,-0.025742,0.085374,-0.025742
3,0.018068,0.035667,4.217599,0.035667,0.072274,0.023351,0.131918,-0.094635,0.219864,0.219864,...,0.090342,-0.094635,-0.094635,4.217599,-0.075708,0.028534,0.021400,0.028534,-0.094635,0.028534
4,0.002185,0.004314,-0.094635,0.004314,0.008742,0.002824,0.015956,4.988553,0.026594,0.026594,...,0.010927,4.988553,4.988553,-0.094635,3.990843,0.003451,0.002588,0.003451,4.988553,0.003451


In [44]:
# Average PREDICTED rating for each item
preds_df.mean().head()

productId
1400599997    0.088513
B00000DM9M    0.449816
B00000J061    0.558292
B00000J08C    0.449816
B00000J0A2    0.354053
dtype: float64

In [45]:
rmse_df = pd.concat([final_ratings_matrix.mean(), preds_df.mean()], axis=1)
rmse_df.columns = ['Avg_actual_ratings', 'Avg_predicted_ratings']
print(rmse_df.shape)
rmse_df['item_index'] = np.arange(0, rmse_df.shape[0], 1)
rmse_df.head()

(186, 2)


,Avg_actual_ratings,Avg_predicted_ratings,item_index
productId,,,
1400599997,0.090909,0.088513,0
B00000DM9M,0.454545,0.449816,1
B00000J061,0.454545,0.558292,2
B00000J08C,0.454545,0.449816,3
B00000J0A2,0.363636,0.354053,4


In [46]:
RMSE = round((((rmse_df.Avg_actual_ratings - rmse_df.Avg_predicted_ratings) ** 2).mean() ** 0.5), 5)
print('\nRMSE SVD Model = {} \n'.format(RMSE))


RMSE SVD Model = 0.05854 



# 8. Getting top - K ( K = 5) recommendations. 

In [47]:
# Enter 'userID' and 'num_recommendations' for the user #
userID = 9
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)


Below are the recommended items for user(user_id = 9):

                   user_ratings  user_predictions
Recommended Items                                
B00003WGP5                  0.0          0.023351
B00000J061                  0.0          0.023351
B00004RERZ                  0.0          0.023351
B00001QHP5                  0.0          0.023351
B00004RIPE                  0.0          0.023351
